# Experiment 1

In [ ]:
from src import config
from src.analyzers import *
from vaep.transform import StandardScaler, get_df_fitted_mean_std

In [ ]:
import logging
from src.logging import setup_logger
logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 01")

## Load data

- 1000 features (most abundant peptides)
- later a subset of samples is selected

In [ ]:
N_SAMPLES_TO_LOAD = None
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N_07813_M01000'
analysis = AnalyzePeptides(fname=FN_PEPTIDE_INTENSITIES, nrows=N_SAMPLES_TO_LOAD)
analysis.df = analysis.df.sort_index()  # sort by date
assert analysis.df.index.is_unique, "Non-unique training samples"

### Select consecutives samples for training

In [ ]:
import random

N_SAMPLES = 1000
logging.info(f"Selected {N_SAMPLES}")
analysis.N_SAMPLES = N_SAMPLES

def get_consecutive_data_indices(index, n_samples=N_SAMPLES):
    start_sample = len(index) - n_samples
    start_sample = random.randint(0, start_sample)
    return index[start_sample:start_sample+n_samples]


indices_selected = get_consecutive_data_indices(analysis.df.index)
analysis.df = analysis.df.loc[indices_selected]
analysis.df

### Create meta data from filename

In [ ]:
from collections import namedtuple

columns = 'date ms_instrument columntype researcher rest'.split()

RunMetaData = namedtuple('RunMetaData', columns)
data_meta = {}
for filename in indices_selected:
    # ToDo: this approach is too easy for the moment. The first two fields are in order, the rest needs matching.
    _meta_filename = filename.split('_', maxsplit=4)
    data_meta[filename] = _meta_filename
    # print(RunMetaData(*_meta_filename[:6]))

analysis.df_meta = pd.DataFrame.from_dict(
    data_meta, orient='index', columns=columns)
# analysis.df_meta['date'] = pd.to_datetime(analysis.df_meta['date'])
analysis.df_meta

In [ ]:
FRACTION = 0.9

class Indices(SimpleNamespace):
    pass

indices = Indices()
indices.train, indices.val = indices_selected[:int(
    FRACTION*N_SAMPLES)], indices_selected[int(FRACTION*N_SAMPLES):]
analysis.indices = indices

analysis.df_meta.loc[indices.train].describe(datetime_is_numeric=False)

In [ ]:
# This becomes part of analysis
def compare_meta_data_for_splits(meta, indices):

    _indices = vars(indices)
    logging.info('Found vars: {}'.format(', '.join(str(x)
                                                   for x in _indices.keys())))

    for key_split, split in _indices.items():
        print(f"{key_split:8} - split description:")
        display(
            meta.loc[split].describe(datetime_is_numeric=True)
        )

    _meta_features = list(meta.columns)

    for _column in _meta_features:
        display(
            _ = pd.DataFrame({
                key_split: meta.loc[split, _column].value_counts(normalize=True) for key_split, split in _indices.items()
            }).sort_index().plot(kind='line', rot=90, figsize=(10,5), title=f"{_column} value Counts for different splits")
        )
compare_meta_data_for_splits(analysis.df_meta.iloc[:,:2], indices)

## Transforms

### Custom Transforms

- illustrate using adapted scikit-learn [`StandardScaler`](https://scikit-learn.org/stable/modules/preprocessing.html)

In [ ]:
N, M = 10, 10  # Samples, Features
analysis.df.iloc[:N, :M]

In [ ]:
analysis.df.iloc[:, :M].describe()

### StandardScaler on raw data

In [ ]:
scaler = StandardScaler().fit(analysis.df)
scaler_df = get_df_fitted_mean_std(scaler, index=analysis.df.columns)
scaler_df.head(N)

In [ ]:
sample = scaler.transform(analysis.df.iloc[:N])
sample.iloc[:, :M]

In [ ]:
sample = scaler.inverse_transform(sample)
sample.iloc[:, :M]

### StandardScaler on log10 transformed data

In [ ]:
X_log10 = np.log10(analysis.df)
X_log10.iloc[:N, :M]

In [ ]:
scaler_log = StandardScaler(
).fit(X=X_log10)
scaler_log_df = get_df_fitted_mean_std(scaler_log, index=analysis.df.index)
scaler_log_df.head(N)

In [ ]:
sample_log10 = scaler_log.transform(X_log10.iloc[:N])
sample_log10.iloc[:, :M]

In [ ]:
scaler_log.inverse_transform(sample_log10).iloc[:, :M]

### Sanity checks

#### Correlation

- Correlation between the computed `means_` should be nearly perfect
- Correlation between peptide intensities should be high
- As taking the logarithm is a monoton, but non-linear transformation, the linear Pearson correlation can change substantially. [[link]](https://stats.stackexchange.com/questions/127121/do-logs-modify-the-correlation-between-two-variables)

In [ ]:
print("Correlation between mean values of linear vs. log-transformed values:",
      f"{np.corrcoef(scaler.mean_, scaler_log.mean_)[1,0]:.4f}", sep='\n')

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format

analysis.corr_linear_vs_log = scaler.transform(X=analysis.df).corrwith(
    other=scaler_log.transform(X_log10),
    axis=0)
analysis.corr_linear_vs_log.describe()

In [ ]:
# own implemention could be slightly faster as data is already demeanded and standardized.
# pd.DataFrame.corrwith?

#### Distribution

In [ ]:
import seaborn as sns
from vaep.utils import sample_iterable

columns_sampled = sample_iterable(list(analysis.df.columns), n=12)
print(columns_sampled)

In [ ]:
def plot_scaled_sample(columns_sampled: list, scaler, df: pd.DataFrame = analysis.df):
    _scaled = scaler.transform(df)
    display(_scaled.describe())
    _min, _max = _scaled.min().min(), _scaled.max().max()
    return _min, _max
    print(list(range(_min, _max, step=0.5)))


_min, _max = plot_scaled_sample(columns_sampled=columns_sampled, scaler=scaler)

In [ ]:
# if bins should be equal between plots
# addon
import math
xlim = [-5, 5]
FACTOR = 1
[x/FACTOR for x in range(math.floor(xlim[0])*FACTOR,
                         math.ceil(xlim[1])*FACTOR+1)]

In [ ]:
import matplotlib.pyplot as plt

columns_sampled = sample_iterable(list(analysis.df.columns), n=9)
subplot_kw = {'xlim': [-5, 5], 'ylim': [0, 600]}
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(
    15, 15), sharey=True, sharex=True, subplot_kw=subplot_kw)
_ = scaler_log.transform(X_log10)[columns_sampled].hist(
    figsize=(15, 15), ax=axes)
axes = scaler.transform(analysis.df)[columns_sampled].hist(
    figsize=(15, 15), ax=axes)
_ = fig.legend(('linear', 'log'), loc=5)

In [ ]:
caption = (f"Frequencies are capped at {subplot_kw['ylim'][1]} and "
           "their standardized intensity values plotted between {} and {}.".format(
               *subplot_kw['xlim'])
           )
print(caption)

## Fastai Dataloader

> fastai includes a replacement for Pytorch's DataLoader which is largely API-compatible, and adds a lot of useful functionality and flexibility. Before we look at the class, there are a couple of helpers we'll need to define. [[link](https://docs.fast.ai/data.load.html)]

In [ ]:
import fastai.tabular.all as tab

In [ ]:
from fastai.tabular.data import TabularDataLoaders

Create dataloaders using an appropriate factory method from `TabularDataLoaders` class, here [`from_df`](https://docs.fast.ai/tabular.data.html#TabularDataLoaders.from_df)

In [ ]:
# DataFrame is shuffled
N_VAL = 100
valid_idx = list(range(N_VAL))
dls = TabularDataLoaders.from_df(df=analysis.df, valid_idx=valid_idx, bs=64)
analysis.dls = dls

In [ ]:
dls.show_batch()  # loses object index attribute

In [ ]:
dls.valid.show_batch()

In [ ]:
for batch in dls.train:
    print(batch)
    break

In [ ]:
# Normalize

So now the `loss_func` signature and the `NN_Module` forward path have to be adapted. Unsure how to do this in plain PyTorch yet. So we only use the dataloader for now.

- Callback needed to set `xb` to `yb`, see [callback-attributes](https://docs.fast.ai/callback.core.html#Attributes-available-to-callbacks) and [example](https://github.com/dhuynh95/fastai_autoencoder/blob/bc357927f26273d676dca9a41018411408b97430/fastai_autoencoder/callback.py#L16)

In [ ]:
# loss_function(recon_x=recon_batch, x=batch, mask=mask, mu=mu, logvar=logvar)
# learn = Learner(dls, NN_Module, opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

## PyTorch

### Setup

In [ ]:
from datetime import datetime

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

# import importlib; importlib.reload(vaep.model)
from vaep.model import train
from vaep.model import VAE
from vaep.model import loss_function
from vaep.cmd import get_args

from vaep.io.datasets import PeptideDatasetInMemory

In [ ]:
args = get_args(no_cuda=True)
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")
device

In [ ]:
from collections import namedtuple

# EpochAverages = namedtuple("EpochAverages", 'loss mse kld')


def eval(epoch, model, data_loader, device, writer=None):
    model.eval()
    metrics = {'loss': 0, 'mse': 0,  'kld': 0}

    for batch, mask in data_loader:
        recon_batch, mu, logvar = model(batch)
        loss, mse, kld = loss_function(
            recon_x=recon_batch, x=batch, mask=mask, mu=mu, logvar=logvar)
        metrics['loss'] += loss.item()
        metrics['mse'] += mse.item()
        metrics['kld'] += kld.item()
    if writer is not None:
        n_samples = len(data_loader.dataset)
        writer.add_scalar('avg validation loss',
                          metrics['loss'] / n_samples,
                          epoch)
    return metrics

In [ ]:
detection_limit = np.log10(analysis.df).min().min()  # all zeros become nan.
"Detection limit: {:6.3f}, corresponding to intensity value of {:,d}".format(
    detection_limit,
    int(10 ** detection_limit)
)

### Training and Validation datasets

In [ ]:
n_samples, n_features = analysis.df.shape
"N samples: {:10,d} - N Peptides: {:10,d}".format(n_samples, n_features)

In [ ]:
# random sample is drawn
PROPORTION_TRAIN = 0.9

# could be a method on analysis
analysis.indices_train = analysis.df.sample(
    int(n_samples*PROPORTION_TRAIN)
).index
analysis.indices_val = analysis.df.index.difference(analysis.indices_train)

analysis.df_train = analysis.df.loc[analysis.indices_train]
analysis.df_val = analysis.df.loc[analysis.indices_val]

len(analysis.indices_val), analysis.indices_val[:5]

### Scaling non-log transformed data

Scale samples according to training data

In [ ]:
# select initial data: transformed vs not log transformed
scaler = scaler.fit(analysis.df_train)
# five examples from validation dataset
scaler.transform(analysis.df_val.iloc[:5])

#### Dataloaders

In [ ]:
data_train = PeptideDatasetInMemory(data=scaler.transform(analysis.df_train))
data_valid = PeptideDatasetInMemory(data=scaler.transform(analysis.df_val))

dl_train = torch.utils.data.DataLoader(
    dataset=data_train,
    batch_size=args.batch_size, shuffle=True, **kwargs)

dl_val = torch.utils.data.DataLoader(
    dataset=data_valid,
    batch_size=args.batch_size, shuffle=False, **kwargs)

print(
    "N train: {:5,d} \nN valid: {:5,d}".format(
        len(dl_train.dataset), len(dl_val.dataset))
)


data, mask = next(iter(dl_train))

prefix_fn = 'experiment_01'
logdir = f'runs/{prefix_fn}_{format(datetime.now(), "%y%m%d_%H%M")}'
writer = SummaryWriter(logdir)
writer.add_image(
    f'{len(data)} batch of sampled data (as heatmap)', data, dataformats='HW')
writer.add_image(
    f'{len(mask)} mask for this batch of samples', mask, dataformats='HW')


n_neurons = max(30, int(n_features/6))
print(f'Latent layer neurons: {n_neurons}')

model = VAE(n_features=n_features, n_neurons=n_neurons).double()
# model.eval()
print(model)
model = model.to(device, non_blocking=True)
writer.add_graph(model, input_to_model=data)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

#### Training Loop

In [ ]:
losses = {}
evaluation = {}
msg_eval_epoch = "Validation Set - Epoch: {:3d} - loss: {:7.3f} - mse: {:5.3f} - KLD: {:5.3f}"

# args.epochs = 10; args.epochs

for epoch in range(1, args.epochs):
    losses[epoch] = train(epoch, model=model, train_loader=dl_train,
                          optimizer=optimizer, device=device, writer=writer)
    evaluation[epoch] = eval(
        epoch, model=model, data_loader=dl_val, device=device, writer=writer)
    if not epoch % 10:
        print(msg_eval_epoch.format(epoch, *evaluation[epoch].values()))
writer.flush()
writer.close()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

-

In [ ]:
# first time, it timesout, second time it starts, see https://github.com/tensorflow/tensorboard/issues/2481#issuecomment-516819768
%tensorboard --logdir {logdir} --host localhost

## Hyperparameter comparison

- [x] order data by date: consecutive samples from training to validation
- [ ] stratify data based on machine and column length (from filename)
- [ ] compare performance regarding data normalization
    - in original intensity space (non-log-transformed)
- [ ] compare performance regarding several hyperparameters of VAE (layers, activation, etc)
    - plot different losses in one plot as validation data set is the same
- [ ] increase number of samples in training set and create result plot


- Current optimum for comparision is zero

> The comparison where relatively low abundant, but not super low-abundant peptides will be masked, could skew the comparison.